# Data Normalization: *Part 1*
#### **Extreme Outlier Removal**: Analyzing relative extremes across Primary Activites, z-Score analysis, IQR (Identifying outlier "bins")

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import f_oneway
from scipy.stats import zscore
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

data = pd.read_csv('merged_data.csv')

# create dataframe
data['Scope_3_emissions_amount'] = pd.to_numeric(data['Scope_3_emissions_amount'], errors='coerce')
# remove nulls
data = data.dropna(subset=['Scope_3_emissions_amount', 'account_id'])
# remove scientific notation
pd.set_option('display.float_format', lambda x: '%.2f' % x)  

data.head()

In [ ]:
# count number of rows less than or equal to 0
zeros = (data['Scope_3_emissions_amount']<= 0).sum()
print("Total Zeros:  " + str(zeros))

In [ ]:
# Create new dataframe with rows where column value <= 0
new_df = data[data['Scope_3_emissions_amount'] <= 0]

# Check how many rows meet this condition:
print("Number of rows where column_name <= 0:", len(new_df))
print("Percentage of total rows:", (len(new_df)/len(data))*100, "%")
print("\nValue counts:")
print(new_df['Scope_3_emissions_amount'].value_counts().sort_index())

In [ ]:
# Create new dataframe with rows where column value <= 0
data_clean = data[data['Scope_3_emissions_amount'] > 0]

# To see how many rows meet this condition:
print("Number of rows where column_name <= 0:", len(data_clean))
print("Percentage of total rows:", (len(data_clean)/len(data))*100, "%")

data_clean

In [ ]:
# print column names
data_clean.columns.unique()

In [ ]:
# create a dataframe with select columns
outlier_df = data_clean[['account_name'
                         , 'account_id'
                         , 'Year'
                         , 'incorporated_country'
                         , 'Primary activity'
                         , 'Primary sector'
                         , 'Market_Cap_USD'
                         , 'Third_party_verification'
                         , 'Revenue_USD'
                         , 'ebitda_USD'
                         , 'grossProfit_USD'
                         , 'netIncome_USD'
                         , 'cashAndCashEquivalents_USD'
                         , 'shortTermInvestments_USD'
                         , 'longTermInvestments_USD'
                         , 'totalAssets_USD'
                         , 'totalLiabilities_USD'
                         , 'totalInvestments_USD'
                         , 'totalDebt_USD'
                         , 'totalEquity_USD'
                         , 'country_gdp'
                         , 'country_total_ghg'
                         , 'country_population'
                         , 'Scope_3_emissions_type'
                         , 'Scope_3_emissions_amount']] 
outlier_df

In [ ]:
# Check for null scope 3 emissions observations
outlier_df['Scope_3_emissions_amount'].isnull().sum()

## EDA and Normalization

##### *Capture Scope 3 Amount Skewness & Kurtosis Before Outlier Removal*

In [ ]:
# Calculate skewness and kurtosis
original_skew = outlier_df['Scope_3_emissions_amount'].skew()
original_kurtosis = outlier_df['Scope_3_emissions_amount'].kurtosis()
print("Skew: ") 
print(original_skew)
print("Kurtosis: ") 
print(original_kurtosis)

#### *Observation Volume*

The GHG data includes data entries from 2013 to 2023, however, not all companies have reported every single year. In order to have sufficinet data, I am going to narrow the years down to a range with the most volume and exclude accounts that do not report for all of those range years.

In [ ]:
# Analyze the amount of records for each year
outlier_df.groupby('Year').size().plot(kind='bar')
plt.title('Unique Rows by Year')
plt.ylabel('Count')
plt.show()

As suspected, the number of entreies grows each year. Given this, I will keep the accounts with records in every year from 2018 to 2023. This will allow for 5 years of training data to predict the final year (2023).

In [ ]:
# Get account_ids that appear in all years
valid_accounts = outlier_df.groupby('account_id')['Year'].unique().apply(set)
valid_accounts = valid_accounts[valid_accounts.apply(lambda x: all(year in x for year in range(2018, 2024)))].index
valid_accounts.shape

In [ ]:
# Filter the dataframe to keep only those account_ids
outlier_df = outlier_df[outlier_df['account_id'].isin(valid_accounts)]
outlier_df

In [ ]:
outlier_df['Year'].nunique()

In [ ]:
# verify final result
outlier_df = outlier_df[(outlier_df['Year'] >= 2018)]
print(outlier_df['Year'].unique())

In [ ]:
# how large is the dataset
outlier_df.shape

## EDA & Normalization : Extreme Variation

Now that the data has been narrowed down to a smaller scope of, I want to look for extreme variation in account reporting year over year within a primary activity. Through the process of EDA in Tableau, it was discovered that some accounts report extremely high and extremely low numbers for the same primary activity across different years. Because it is impossible to validate any level of accuracy in this instance, I believe these accounts should be removed entirely from the data. This function should assist in determining these cases for further analysis and removal. 

In [ ]:
def find_extreme_variations(outlier_df, value_column, year_column='Year', 
                          activity_column='Primary activity', 
                          account_column='account_id',
                          z_score_threshold=2):
    
    # Calculate variation metrics for each account within a primary activity
    variation_stats = (outlier_df.groupby([activity_column, account_column])
                      .agg({
                          value_column: ['std', 'mean', 'min', 'max', 'count'],
                          year_column: list
                      })
                      .reset_index())
    
    # Flatten column names
    variation_stats.columns = [
        f"{col[0]}_{col[1]}" if col[1] else col[0] 
        for col in variation_stats.columns
    ]
    
    # Calculate coefficient of variation (CV)
    variation_stats['cv'] = (variation_stats[f'{value_column}_std'] / 
                           variation_stats[f'{value_column}_mean'].abs())
    
    # Calculate range ratio
    variation_stats['range_ratio'] = (variation_stats[f'{value_column}_max'].abs() / 
                                    variation_stats[f'{value_column}_min'].abs())
    
    # Calculate Z-scores of CV within each Primary Activity
    variation_stats['cv_zscore'] = (variation_stats
                                   .groupby(activity_column)['cv']
                                   .transform(lambda x: stats.zscore(x)))
    
    # Identify extreme accounts
    extreme_accounts = variation_stats[
        (variation_stats['cv_zscore'].abs() > z_score_threshold) &
        (variation_stats[f'{value_column}_count'] > 1)  # At least 2 years of data
    ].copy()
    
    # Sort and format results
    extreme_accounts = extreme_accounts.sort_values(
        ['cv_zscore'], 
        ascending=False
    )
    
    # Add year range information
    extreme_accounts['year_range'] = extreme_accounts[f'{year_column}_list'].apply(
        lambda x: f"{min(x)}-{max(x)}"
    )
    
    return extreme_accounts

In [ ]:
# run function
results = find_extreme_variations(
    outlier_df,
    value_column='Scope_3_emissions_amount',
    z_score_threshold=2
)
print(results)

In [ ]:
# For every account and primary activity combo where extreme variation was found, plot the YoY trend:

def create_plots(df, results):
    # Filter dataframe to only include accounts that are in results
    filtered_df = outlier_df[outlier_df['account_id'].isin(results['account_id'])]
    
    # Get unique combinations of account_id and Primary activity
    unique_combinations = filtered_df[['account_id', 'Primary activity']].drop_duplicates()
    
    # Create a separate plot for each combination
    for _, row in unique_combinations.iterrows():
        account = row['account_id']
        activity = row['Primary activity']
        
        # Filter data for this specific combination
        plot_data = filtered_df[
            (filtered_df['account_id'] == account) & 
            (filtered_df['Primary activity'] == activity)
        ]
        
        # Create individual plot
        plt.figure(figsize=(4, 2))
        sns.barplot(
            data=plot_data,
            x='Year',
            y='Scope_3_emissions_amount',
            errorbar=None
        )
        
        # Customize plot
        plt.title(f'Account: {account}\nPrimary Activity: {activity}')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

# Run function
create_plots(outlier_df, results)

In [ ]:
results.shape

In [ ]:
# Looking at each of these trends, it's clear there is very inconsistent reporting YoY and these should be excluded
# Remove all 64 accounts that were found in the extreme variation function
outlier_df = outlier_df[~outlier_df['account_id'].isin(results['account_id'])]
outlier_df

## EDA and Normalization: Z-Score Analysis

Now that accounts with unreliable/extreme variation have been removed, I will create a function to detect outliers within each Primary activity that falls above a z-score threshold of 3 to see what outliers remain.

In [ ]:
# Initialize list to store outlier records
outlier_records = []

# Process each Primary Activity
for activity in outlier_df['Primary activity'].unique():
    # Get data for this activity
    activity_data = outlier_df[outlier_df['Primary activity'] == activity]
    
    # Calculate z-scores and std dev for this activity
    z_scores = stats.zscore(activity_data['Scope_3_emissions_amount'])
    std_dev = activity_data['Scope_3_emissions_amount'].std()
    
    # Find outliers (z-score >= 3)
    outlier_mask = abs(z_scores) >= 3
    
    # Get outlier records
    outliers = activity_data[outlier_mask].copy()
    outliers['z_score'] = z_scores[outlier_mask]
    outliers['std_dev'] = std_dev
    
    # Add to records list
    outlier_records.append(outliers[['account_id', 'account_name', 'Primary activity', 
                                   'Scope_3_emissions_amount', 'Year', 'z_score', 'std_dev']])

# Combine all outliers into one dataframe
outliers = pd.concat(outlier_records)

# Sort by absolute z-score (highest to lowest)
outliers = outliers.sort_values(by='z_score', key=abs, ascending=False)

# Reset index
outliers = outliers.reset_index(drop=True)

print("\nOutliers Detail:")
print(outliers)

In [ ]:
# Vizualize the outliers YoY
# Create the scatter plot
fig = px.scatter(outliers, 
                 x='Year', 
                 y='z_score',
                 color='Primary activity',
                 hover_data={
                     'account_name': True,
                     'Scope_3_emissions_amount': ':.2f',
                     'z_score': ':.2f',
                     'Year': True
                 },
                 title='Scope 3 Emissions Outliers Over Time')

# Add horizontal lines for z-score thresholds
fig.add_hline(y=3, line_dash="dash", line_color="red", annotation_text="z-score = 3")
fig.add_hline(y=-3, line_dash="dash", line_color="red")

# Update layout
fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Z-Score",
    height=600,
    width=1000,
    showlegend=True
)

# Show the plot
fig.show()

Since most of the remaining 985 outliers fall under a z-score of 15, I will first remove accounts with a z-score of 15 or higher.

In [ ]:
# Group by Primary activity and re-calculate z-scores within each group
z_scores = outlier_df.groupby('Primary activity')['Scope_3_emissions_amount'].transform(
    lambda x: abs((x - x.mean()) / x.std())
)

# Find accounts that have any z-score of 15 or higher
outlier_accounts = outlier_df[z_scores >= 15]['account_id'].unique()

# Remove all rows for these accounts
outlier_df_cleaned = outlier_df[~outlier_df['account_id'].isin(outlier_accounts)].copy()

# Print summary
total_removed = len(outlier_df) - len(outlier_df_cleaned)
accounts_removed = len(outlier_accounts)
print(f"Total accounts removed: {accounts_removed}")
print(f"Total rows removed: {total_removed}")

# Show removal count by Primary activity
removal_by_activity = (
    outlier_df['Primary activity'].value_counts() - 
    outlier_df_cleaned['Primary activity'].value_counts()
).fillna(0)
print("\nRows removed by Primary activity:")
print(removal_by_activity[removal_by_activity > 0])

This removed 8 more accounts and a total of 320 rows. I will vizualize the final results looking at a scatterplot of every value and which percentile they land in to better understand the distribution. 

In [ ]:
# calculate the percentiles
percentiles = [25, 50, 80, 90, 95, 96, 97, 98, 99, 100]
percentile_values = {p: outlier_df_cleaned['Scope_3_emissions_amount'].quantile(p/100) for p in percentiles}

# Create color scale for different percentile ranges
color_scale = ['#d4e6f1','#a9cce3','#7fb3d5','#5499c7','#2980b9', '#2471a3', '#1f618d', '#1a5276', '#1b4f72', '#641e16']

# Initialize the figure
fig = go.Figure()
#1b4f72
# Create percentile ranges and add traces for each range
prev_percentile = 0
for i, percentile in enumerate(percentiles):
    if i == 0:
        mask = outlier_df_cleaned['Scope_3_emissions_amount'] <= percentile_values[percentile]
        start = 0
    else:
        mask = (outlier_df_cleaned['Scope_3_emissions_amount'] > percentile_values[prev_percentile]) & \
               (outlier_df_cleaned['Scope_3_emissions_amount'] <= percentile_values[percentile])
        start = prev_percentile
    
    data = outlier_df_cleaned[mask].copy()
    data['percentile_range'] = f"{start}-{percentile}"
    
    if len(data) > 0:  # Only add trace if there is data in this range
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=data['Scope_3_emissions_amount'],
                mode='markers',
                name=f"{start}-{percentile}th percentile",
                marker=dict(
                    color=color_scale[i] if i < len(color_scale) else color_scale[-1],
                    size=6
                ),
                hovertemplate=(
                    '<b>Account ID:</b> %{customdata[0]}<br>' +
                    '<b>Account Name:</b> %{customdata[1]}<br>' +
                    '<b>Primary Activity:</b> %{customdata[2]}<br>' +
                    '<b>Scope 3 Emissions:</b> %{y:,.2f}<br>' +
                    '<b>Percentile Range:</b> %{customdata[3]}<br>'
                ),
                customdata=np.column_stack((
                    data['account_id'],
                    data['account_name'],
                    data['Primary activity'],
                    data['percentile_range']
                ))
            )
        )
    prev_percentile = percentile

# Add lines for each percentile
for percentile, value in percentile_values.items():
    fig.add_hline(
        y=value,
        line_dash="dash",
        line_color="gray",
        line_width=1,
        annotation=dict(
            text=f"{percentile}th percentile",
            xref="paper",
            yref="y",
            x=1.02,
            y=value,
            showarrow=False,
            font=dict(size=8)
        )
    )

# Update layout
fig.update_layout(
    height=800,
    width=1200,
    title_text="Scope 3 Emissions Distribution by Percentile Ranges",
    xaxis_title="Index",
    yaxis_title="Scope 3 Emissions Amount",
    showlegend=True,
    legend_title="Percentile Ranges",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.02,
        bgcolor="rgba(255, 255, 255, 0.8)"
    )
)

# Show the plot
fig.show()

# Print summary statistics
print("\nSummary Statistics:")
for p in percentiles:
    print(f"{p}th percentile: {percentile_values[p]:,.2f}")
print(f"\nTotal number of observations: {len(outlier_df_cleaned)}")
print(f"Number of unique accounts: {outlier_df_cleaned['account_id'].nunique()}")

In [ ]:
# How many observations fall into each of the percentiles?

# Initialize dictionary to store counts
counts = {}

# Calculate for first bin (0 to 25th percentile)
mask = outlier_df_cleaned['Scope_3_emissions_amount'] <= percentile_values[25]
counts['0-25'] = len(outlier_df_cleaned[mask])

# Calculate for intermediate bins
for i in range(len(percentiles)-1):
    current_percentile = percentiles[i]
    next_percentile = percentiles[i+1]
    
    mask = (outlier_df_cleaned['Scope_3_emissions_amount'] > percentile_values[current_percentile]) & \
           (outlier_df_cleaned['Scope_3_emissions_amount'] <= percentile_values[next_percentile])
    
    counts[f'{current_percentile}-{next_percentile}'] = len(outlier_df_cleaned[mask])

# Print results with percentages
total_observations = len(outlier_df_cleaned)
print("\nObservations in each percentile range:")
print("-" * 60)
print(f"{'Percentile Range':<20} {'Count':>10} {'Percentage':>12}")
print("-" * 60)

for range_name, count in counts.items():
    percentage = (count / total_observations) * 100
    print(f"{range_name:<20} {count:>10,} {percentage:>11.2f}%")

print("-" * 60)
print(f"{'Total':<20} {total_observations:>10,} {100:>11.2f}%")

There remains significant jump in the 95th percentile. I will vizualize the z-scores within each primary activity.

In [ ]:
# Get unique primary activities
activities = outlier_df_cleaned['Primary activity'].unique()

# Calculate number of rows and columns for subplots
n_plots = len(activities)
n_cols = 3
n_rows = (n_plots + n_cols - 1) // n_cols

# Create subplots
fig, axs = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
axs = axs.flatten()

# For loop that will create a plot for each primary activity
for idx, activity in enumerate(activities):
    # Get data for activity
    activity_data = outlier_df[outlier_df['Primary activity'] == activity]
    
    # Calculate z-scores for this activity
    z_scores = stats.zscore(activity_data['Scope_3_emissions_amount'])
    
    # Create scatter plot
    axs[idx].scatter(activity_data['Year'], z_scores, alpha=0.5)
    
    # Add threshold line
    axs[idx].axhline(y=3, color='r', linestyle='--', label='Z-score = 3')
    axs[idx].axhline(y=-3, color='r', linestyle='--')
    
    # Customize plot
    axs[idx].set_title(f'Primary Activity: {activity}')
    axs[idx].set_xlabel('Year')
    axs[idx].set_ylabel('Z-score')
    axs[idx].grid(True)
    axs[idx].legend()

# Remove empty subplots if any
for idx in range(len(activities), len(axs)):
    fig.delaxes(axs[idx])

plt.tight_layout()
plt.show()

There are still some significant outliers within almost every primary activity, so it is not a limited set of activities causing skew. I will do a quick binning to see how many outliers fall in different thresholds.

In [ ]:
# Initialize results storage
results = []

# Loop through each Primary Activity
for activity in outlier_df['Primary activity'].unique():
    # Get data for this activity
    activity_data = outlier_df[outlier_df['Primary activity'] == activity]
    
    # Calculate z-scores
    z_scores = stats.zscore(activity_data['Scope_3_emissions_amount'])
    
    # Count outliers in different ranges
    low_outliers = sum((z_scores >= 3) & (z_scores < 5))
    high_outliers = sum((z_scores >= 5) & (z_scores < 8))
    extreme_outliers = sum(z_scores >= 8)
    
    # Store results
    results.append({
        'Primary Activity': activity,
        'Outliers (3 to 5)': low_outliers,
        'Outliers (5 to 8)': high_outliers,
        'Outliers (8 or higher)': extreme_outliers,
        'Total Outliers': low_outliers + high_outliers + extreme_outliers
    })

# Convert to DataFrame
results_df = pd.DataFrame(results)

# Calculate totals row
totals = {
    'Primary Activity': 'TOTAL',
    'Outliers (3 to 5)': results_df['Outliers (3 to 5)'].sum(),
    'Outliers (5 to 8)': results_df['Outliers (5 to 8)'].sum(),
    'Outliers (8 or higher)': results_df['Outliers (8 or higher)'].sum(),
    'Total Outliers': results_df['Total Outliers'].sum()
}

# Append totals row
results_df = pd.concat([results_df, pd.DataFrame([totals])], ignore_index=True)

print("\nOutlier Counts by Primary Activity:")
print(results_df.to_string(index=False))

Histogram and stats to look at the distribution.

In [ ]:
# Create the histogram
sns.histplot(data=outlier_df_cleaned, x='Scope_3_emissions_amount', bins=40, kde=True)
# Customize the plot
plt.title('Distribution of Amount')
plt.xlabel('Amount')
plt.ylabel('Frequency')
# Add grid for better readability
plt.grid(True, alpha=0.3)
# Show the plot
plt.show()

# final df stats
print(outlier_df_cleaned['Scope_3_emissions_amount'].describe())

In [ ]:
# how many rows are above the 98th percentile and what is the value of that percentile?
column_name = 'Above_98'
percentile_98 = outlier_df_cleaned['Scope_3_emissions_amount'].quantile(0.98)
count_above_98 = outlier_df_cleaned[outlier_df_cleaned['Scope_3_emissions_amount'] > percentile_98].shape[0]
count_rows = outlier_df_cleaned['Scope_3_emissions_amount'].notna().sum()
percent = count_above_98/count_rows*100

print(f"98th percentile value: {percentile_98}")
print("Total Rows in Data: " + str(count_rows))
print(f"Number of rows above 98th percentile: {count_above_98}")
print(f"% above 98th percentile: {percent}")

In [ ]:
# how many rows are above the 99th percentile and what is the value of that percentile?
column_name = 'Above_99'
percentile_99 = outlier_df_cleaned['Scope_3_emissions_amount'].quantile(0.99)
count_above_99 = outlier_df_cleaned[outlier_df_cleaned['Scope_3_emissions_amount'] > percentile_99].shape[0]
count_rows = outlier_df_cleaned['Scope_3_emissions_amount'].notna().sum()
percent = count_above_99/count_rows*100

print(f"99th percentile value: {percentile_99}")
print("Total Rows in Data: " + str(count_rows))
print(f"Number of rows above 99th percentile: {count_above_99}")
print(f"% above 99th percentile: {percent}")

In [ ]:
# how many accounts have at least 1 row with a scope 3 emissions amount in the 99th percentile or higher
# Calculate the overall 99th percentile
overall_99th = outlier_df_cleaned['Scope_3_emissions_amount'].quantile(0.99)

# Find accounts with at least one emission value above 99th percentile
high_emission_accounts_99 = outlier_df_cleaned[
    outlier_df_cleaned['Scope_3_emissions_amount'] >= overall_99th
]['account_id'].unique()

# Count these accounts
n_high_accounts = len(high_emission_accounts_99)

print(f"Number of accounts with emissions in 99th percentile or higher: {n_high_accounts}")
print(f"Percentage of total accounts: {(n_high_accounts/outlier_df_cleaned['account_id'].nunique()*100):.1f}%")
print(f"\n99th percentile threshold: {overall_99th:,.2f}")

The extreme positive skew in the data is difficult to correct for without . The data must always be analyzed with consideration of Primary activity, as extreme differences in each category is expected. However, I think it is best to remove any account that has observations in the 99th percentile or higher since 52 million is not likely to be a valid amount.

In [ ]:
# remove all accounts that have at least 1 value in the 99th percentile
# Find accounts with at least one emission value above 99th percentile
accounts_to_remove = outlier_df_cleaned[
    outlier_df_cleaned['Scope_3_emissions_amount'] >= overall_99th
]['account_id'].unique()

# Create new dataframe excluding these accounts
filtered_df = outlier_df_cleaned[~outlier_df_cleaned['account_id'].isin(accounts_to_remove)]

print(f"Original number of accounts: {outlier_df_cleaned['account_id'].nunique()}")
print(f"Number of accounts removed: {len(accounts_to_remove)}")
print(f"Remaining number of accounts: {filtered_df['account_id'].nunique()}")
print(f"\n99th percentile threshold: {overall_99th:,.2f}")

I want to re-visualize the data with these accounts removed

In [ ]:
# calculate the percentiles
percentiles = [25, 50, 80, 90, 95, 96, 97, 98, 99, 100]
percentile_values = {p: filtered_df['Scope_3_emissions_amount'].quantile(p/100) for p in percentiles}

# Create color scale for different percentile ranges
color_scale = ['#d4e6f1','#a9cce3','#7fb3d5','#5499c7','#2980b9', '#2471a3', '#1f618d', '#1a5276', '#1b4f72', '#641e16']

# Initialize the figure
fig = go.Figure()
#1b4f72
# Create percentile ranges and add traces for each range
prev_percentile = 0
for i, percentile in enumerate(percentiles):
    if i == 0:
        mask = filtered_df['Scope_3_emissions_amount'] <= percentile_values[percentile]
        start = 0
    else:
        mask = (filtered_df['Scope_3_emissions_amount'] > percentile_values[prev_percentile]) & \
               (filtered_df['Scope_3_emissions_amount'] <= percentile_values[percentile])
        start = prev_percentile
    
    data = filtered_df[mask].copy()
    data['percentile_range'] = f"{start}-{percentile}"
    
    if len(data) > 0:  # Only add trace if there is data in this range
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=data['Scope_3_emissions_amount'],
                mode='markers',
                name=f"{start}-{percentile}th percentile",
                marker=dict(
                    color=color_scale[i] if i < len(color_scale) else color_scale[-1],
                    size=6
                ),
                hovertemplate=(
                    '<b>Account ID:</b> %{customdata[0]}<br>' +
                    '<b>Account Name:</b> %{customdata[1]}<br>' +
                    '<b>Primary Activity:</b> %{customdata[2]}<br>' +
                    '<b>Scope 3 Emissions:</b> %{y:,.2f}<br>' +
                    '<b>Percentile Range:</b> %{customdata[3]}<br>'
                ),
                customdata=np.column_stack((
                    data['account_id'],
                    data['account_name'],
                    data['Primary activity'],
                    data['percentile_range']
                ))
            )
        )
    prev_percentile = percentile

# Add lines for each percentile
for percentile, value in percentile_values.items():
    fig.add_hline(
        y=value,
        line_dash="dash",
        line_color="gray",
        line_width=1,
        annotation=dict(
            text=f"{percentile}th percentile",
            xref="paper",
            yref="y",
            x=1.02,
            y=value,
            showarrow=False,
            font=dict(size=8)
        )
    )

# Update layout
fig.update_layout(
    height=800,
    width=1200,
    title_text="Scope 3 Emissions Distribution by Percentile Ranges",
    xaxis_title="Index",
    yaxis_title="Scope 3 Emissions Amount",
    showlegend=True,
    legend_title="Percentile Ranges",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.02,
        bgcolor="rgba(255, 255, 255, 0.8)"
    )
)

# Show the plot
fig.show()

# Print summary statistics
print("\nSummary Statistics:")
for p in percentiles:
    print(f"{p}th percentile: {percentile_values[p]:,.2f}")
print(f"\nTotal number of observations: {len(filtered_df)}")
print(f"Number of unique accounts: {filtered_df['account_id'].nunique()}")

In [ ]:
# Look at the histogram again
sns.histplot(data=filtered_df, x='Scope_3_emissions_amount', bins=40, kde=True)
# Customize the plot
plt.title('Distribution of Amount')
plt.xlabel('Amount')
plt.ylabel('Frequency')
# Add grid for better readability
plt.grid(True, alpha=0.3)
# Show the plot
plt.show()

# final df stats
print(filtered_df['Scope_3_emissions_amount'].describe())

#### *Analyze the Change*

In [ ]:
# Calculate the change in standard deviation after outlier removal
std_now = filtered_df['Scope_3_emissions_amount'].std()
std_before = data_clean['Scope_3_emissions_amount'].std()
std_difference = std_now - std_before

print(f"New Standard Deviation: {std_now:,.2f}")
print(f"Original Standard Deviation: {std_before:,.2f}")
print(f"Difference in Standard Deviations: {std_difference:,.2f}")
print("--------------------------")
# Calculate skewness and kurtosis
skew_now = filtered_df['Scope_3_emissions_amount'].skew()
kurtosis_now = filtered_df['Scope_3_emissions_amount'].kurtosis()

# Calculate change
skew_diff = skew_now-original_skew
kurt_diff = kurtosis_now-original_kurtosis

print(f"New Skew: {skew_now:,.2f}")
print(f"Original Skew: {original_skew:,.2f}")
print(f"Difference in Skew: {skew_diff:,.2f}")
print("--------------------------")
print(f"New Kurtosis: {kurtosis_now:,.2f}")
print(f"Original Kurtosis: {original_kurtosis:,.2f}")
print(f"Difference in Kurtosis: {kurt_diff:,.2f}")


While the distribution is still far from normal and extreme positive skewness remains, the standard deviation has been reduced by 328 million from the original dataset and the new skew is 8.03 and the new kurtosis is 77.54 (very high).

This is a significant change to the skew and distribution, but without knowing enough about the underlying data it will take a lot of time and research to fully understand the best way to handle skewness and outliers, whether that is through imputation, more vast removal, industry SME validation, or manual data corrections via data research.

#### *Final Step*

For the final data, rather than having a GDP, GHG, and Population for each year, I am going to create an average so each country has a consistent value for these features.

In [ ]:
# rename back to df
df = filtered_df

# Create an average of all years by country for gdp, ghg, and population
df['country_ghg_avg'] = df.groupby('incorporated_country')['country_total_ghg'].transform('mean')
df['country_population_avg'] = df.groupby('incorporated_country')['country_population'].transform('mean')
df['country_gdp_avg'] = df.groupby('incorporated_country')['country_gdp'].transform('mean')

# Drop original columns
df = df.drop(['country_total_ghg', 'country_population', 'country_gdp'], axis=1)

df.columns.unique()

### Export Data

In [ ]:
# Export to csv
df.to_csv('GHG_Post_Outlier.csv', index=False)

In [ ]:
# compressed csv
df.to_csv('GHG_Post_Outlier.csv.gz', compression='gzip', index=False)